In [ ]:
import pandas as pd
import numpy as np

In [ ]:
merged_survey = pd.read_csv('Merged_survey.csv')

In [ ]:
merged_survey.drop(columns=['RecipientEmail'], inplace=True)

In [ ]:
merged_survey['Treatment'].head()

,Treatment
0,0.0
1,1.0
2,1.0
3,0.0
4,0.0


In [ ]:
merged_survey['Treatment'] = merged_survey['Treatment'].astype(int)

In [ ]:
merged_survey.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 22 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   pre_class_time_spent                        35 non-null     int64  
 1   pre_class_DistributionChannel               35 non-null     object 
 2   CVQ1_gender_bool                            35 non-null     float64
 3   CVQ2_read_stubhub_case_bool                 35 non-null     float64
 4   CVQ3_knowledge_causality_val                35 non-null     float64
 5   CVQ4_likeliness_participate_val             35 non-null     float64
 6   RQ1_understand_material_val                 35 non-null     float64
 7   RQ2_remember_tomorrow_val                   35 non-null     float64
 8   RQ3_active_involvement_val                  35 non-null     float64
 9   Score                                       35 non-null     int64  
 10  Treatment       

In [ ]:
import numpy as np

# Function to calculate Cohen's d
def cohens_d(group1, group2):
    # Calculate the means and standard deviations of both groups
    mean1, mean2 = np.mean(group1), np.mean(group2)
    std1, std2 = np.std(group1, ddof=1), np.std(group2, ddof=1)

    # Pooled standard deviation
    pooled_std = np.sqrt(((len(group1) - 1) * std1**2 + (len(group2) - 1) * std2**2) / (len(group1) + len(group2) - 2))

    # Cohen's d formula
    return (mean1 - mean2) / pooled_std

# Define the response variables
response_vars = ['RQ1_understand_material_val', 'post_class_class_participation_val', 'post_class_recall_val']

# Results storage
cohens_d_results = {}

# Analyze each response variable
for response_var in response_vars:
    control_group = merged_survey[merged_survey['Treatment'] == 0][response_var]
    treatment_group = merged_survey[merged_survey['Treatment'] == 1][response_var]

    # Cohen's d
    cohens_d_results[response_var] = cohens_d(treatment_group, control_group)

cohens_d_results

{'RQ1_understand_material_val': 0.24025492113602034,
 'post_class_class_participation_val': 0.1283647525315742,
 'post_class_recall_val': -0.10772428782883381}

- RQ1_understand_material_val: 0.2403 (small effect)
- post_class_class_participation_val: 0.1283 (small effect)
- post_class_recall_val: -0.1077 (small effect)

In [ ]:
import pingouin as pg
rq1_power = pg.power_ttest(d=cohens_d_results['RQ1_understand_material_val'], n=len(merged_survey), alpha=0.05, alternative='two-sided')
participation_power = pg.power_ttest(d=cohens_d_results['post_class_class_participation_val'], n=len(merged_survey), alpha=0.05, alternative='two-sided')
recall_power = pg.power_ttest(d=cohens_d_results['post_class_recall_val'], n=len(merged_survey), alpha=0.05, alternative='two-sided')
print(f"RQ1 Power: {rq1_power}")
print(f"Participation Power: {participation_power}")
print(f"Recall Power: {recall_power}")

RQ1 Power: 0.16784340907453155
Participation Power: 0.08268141243262343
Recall Power: 0.07290587685255286


In [ ]:
from scipy import stats

import statsmodels.api as sm
import statsmodels.stats.api as sms
count = (merged_survey['Treatment'] == 1).sum()
nobs = len(merged_survey)
stat, pval = sms.proportions_ztest(count, nobs, value = 0.5)
print(f"Z-statistic: {stat:.4f}, P-value: {pval:.4f}")

Z-statistic: -1.5742, P-value: 0.1154


In [ ]:
!pip install pingouin

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.4/204.4 kB 4.3 MB/s eta 0:00:00


In [ ]:
!pip install stargazer

In [ ]:
import statsmodels.formula.api as smf
from stargazer.stargazer import Stargazer

CV1 = smf.ols("CVQ1_gender_bool ~ Treatment", data=merged_survey).fit(cov_type='HC1')
CV2 = smf.ols("CVQ2_read_stubhub_case_bool ~ Treatment", data=merged_survey).fit(cov_type='HC1')
CV3 = smf.ols("CVQ3_knowledge_causality_val ~ Treatment", data=merged_survey).fit(cov_type='HC1')
CV4 = smf.ols("CVQ4_likeliness_participate_val	 ~ Treatment", data=merged_survey).fit(cov_type='HC1')

reg_cv = Stargazer([CV1, CV2, CV3, CV4])
reg_cv